In [1]:
# Installe la bibliothèque pour le rendu du labyrinthe
!pip install pygame

In [2]:
import pygame,sys
from pygame.draw import *
from pygame.display import *
from pygame.locals import * 
from random import randint

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
##################################################################################################
##################################################################################################
##################################################################################################

# Gestion de paramètres de l'écran : taille, couleur de fond
class Screen :
    
    def __init__(self,wid,hei) :
        self.is_running = True
        self.width = wid
        self.height = hei
        self.color = (255,255,255)  # Fond d'écran blanc (à changer si l'on veut)
        self.my_screen = set_mode((wid,hei))
        
        
    # Accesseurs   
    def set_color(self,r,v,b) :
        self.color = r,v,b
    
    def set_image(self,f_name) :
        self.image = pygame.image.load(f_name).convert()
     
    
    # Affichage de l'écran
    def blit(self) :
        self.my_screen.blit(self.image,(0,0))


##################################################################################################
##################################################################################################
##################################################################################################

# Gestion de la fenêtre d'application
class Event :
    
    def __init__(self) :
        return None
    
    # Fermeture de la fenêtre
    def on_close(self) :
        for event in pygame.event.get():
            if event.type == QUIT:       # Si clique sur la croix quitter
                screen.is_running = False
                pygame.display.quit()    # Ferme la fenêtre


##################################################################################################
##################################################################################################
##################################################################################################

# Gestion des noeuds définis par un numéro, des coordonnées, un poids et deux booléens
class Vertex :
    
    def __init__(self,val,abs,ord,wei) :
        self.value = val
        self.coords = (abs,ord)
        self.weight = wei
        self.on_use = False  # Noeud en cours d'utilisation en tant que parent
        self.is_used = False  # Noeud déjà utilisé en tant qu'enfant

        
##################################################################################################
##################################################################################################
##################################################################################################

# Gestion du graphe
class Graph :
    
    def __init__(self,n_vert) :
        self.nb_vertices = n_vert
        self.nodes = []
        self.color = (0,0,0)    # Couleur par défaut pour l'affichage

        
    def init_graph(self) :
        for i in range(self.nb_vertices) :
            self.nodes.append([Vertex(i,0,0,0)])
            
    def add_child(self,par,nbr,abs,ord,nb_wei) :
        self.nodes[par].append(Vertex(nbr,abs,ord,nb_wei))
        
    def delete_children(self,num_par,num_chd) :
        for children in self.nodes :
            if children[0].value == num_par : # On ne supprime pas l'enfant qui est
                continue                      # relié à son parent "optimal"
           
            for i in range(1,len(children)) : # On parcourt seulement les enfants
                if children[i].value == num_chd :  # S'il s'agit de celui cherché,
                    del(children[i])          # on l'efface 
                    break                     # une fois supprimé, on quitte la boucle 
                                       
    
    def all_on_use(self) :   # Indique si un parent n'a pas encore été utilisé
        for children in self.nodes :
            if children[0].on_use == False :
                return False
        
        return True
                
        
    def prim_algorithm(self,val_dep) :
        self.nodes[val_dep][0].on_use = True     # Noeud de départ
        
        # Suppression de tous les enfants correspondant au noeud de départ
        # le '-1' en tant que parent permet de parcourir toute la liste par défaut
        self.delete_children(-1,val_dep)
            
        
        # Tous les parents doivent être traités 
        while self.all_on_use() == False :
        
            min_weight = 1000                         # Stocke le poids minimum
            best_child_num = -1                       # Stocke la valeur de l'enfant le plus proche
            best_parent_num = -1                      # Stocke la valeur du parent dont l'enfant 
                                                      # est le plus proche
        
            for children in self.nodes :
                if children[0].on_use == False :     # Seuls les enfants des parents "on_use"
                    continue                         # sont traités
                
                # L'indice zéro de children est le parent, d'où le range (1,len(children))
                for i in range(1,len(children)) :     # Si on trouve un nouveau chemin intéressant
                    if children[i].weight < min_weight and children[i].is_used == False :     
                        best_child_num = children[i].value   # Nouveau meilleur enfant
                        best_parent_num = children[0].value  # avec son parent
                        min_weight = children[i].weight      # Nouveau poids minimum
            
            # Contrôle sur une erreur éventuelle d'initialisation (oubli d'un lien  ?)        
            assert best_child_num != -1 and best_parent_num != -1,"Pas d'enfant / parent d'indice -1"
        
            self.nodes[best_child_num][0].on_use = True   # Parcours possible d'enfants d'un nouveau parent
            self.delete_children(best_parent_num,best_child_num)  # Suppression des enfants des autres parents
            
            for child in self.nodes[best_parent_num] :  # Désactivation du chemin déjà traité, boucle infinie sinon
                if child.value == best_child_num :
                    child.is_used = True
        
    # Affiche le graphe : chaque parent avec ses enfants + valeur, poids, coordonnées    
    def print_graph(self) :
        for children in self.nodes :
            print("Parent, Val = ", children[0].value, end = " // ")
            for i in range(1,len(children)) :
                print("Val =",children[i].value," - Wei =",children[i].weight,end = ' / ')
            print("\n")

            
##################################################################################################          
##################################################################################################
##################################################################################################

# Gestion du labyrinthe
class Maze :
    
    def __init__(self,scre,lin,col) :
        assert lin >=3 and col >=3,"Indiquer des dimensions de labyrinthe cohérentes"
        self.lines = lin
        self.columns = col
        self.graph = Graph(lin*col)
        self.screen = scre
        self.event = Event()
    
    
    # Accesseurs 
    def set_size(self) :  # "Epaisseur" du labyrinthe, le 0.6 est empirique
        # Choix de l'épaisseur la plus fine si l'écran n'est pas carré
        self.size = min(0.6*self.screen.width/self.columns,0.6*self.screen.height/self.lines)
        
        # Pas entre chaque node en largeur et hauteur en tenant compte
        # du fait que le premier node est à self.size et le dernier self.size du bord
        self.step_width = (self.screen.width - 2*self.size)/(self.columns-1)
        self.step_height = (self.screen.height - 2*self.size)/(self.lines-1)
    
    def set_color(self,col) :
        self.graph.color = col;
        
    def set_max_weight(self,m_wei) :
        self.max_weight = m_wei
        
        
    # Initialisation du labyrinthe    
    def init_maze(self) :
        self.graph.init_graph()
        
        for i in range(self.lines) :
            for j in range(self.columns) :
                value = i*self.columns + j      # Valeur du futur parent
                valueE = value + 1              # Valeur d'un éventuel enfant à droite
                valueS = value + self.columns   # Même chose en bas
                
                # Coordonnées du parent
                par_abs = float(j)*self.step_width + self.size
                par_ord = float(i)*self.step_height + self.size

                self.graph.nodes[value][0].coords = (par_abs,par_ord)
            
                # Création d'un enfant à droite + en dessous si possible
                if j < self.columns - 1: # Si on est n'est pas au bord à droite
                    rand = randint(1,self.max_weight) # Valeurs aléatoires des poids
                    chd_abs = par_abs + self.step_width
                    chd_ord = par_ord
                    self.graph.add_child(value,valueE,chd_abs,chd_ord,rand)
                    
                    # Graphe non-orienté, ajout du lien en retour (échange parent / enfant)
                    self.graph.add_child(valueE,value,par_abs,par_ord,rand)
                    
                if i < self.lines - 1: # Si on n'est pas sur la dernière ligne
                    rand = randint(1,self.max_weight) # Valeurs aléatoires des poids
                    chd_abs = par_abs 
                    chd_ord = par_ord + self.step_height
                    self.graph.add_child(value,valueS,chd_abs,chd_ord,rand)
                    
                    # Graphe non-orienté, ajout du lien en retour (échange parent / enfant)
                    self.graph.add_child(valueS,value,par_abs,par_ord,rand)
    
    
    # Application de l'algorithme de Prim
    def prim_algorithm(self,val_dep) :
        self.graph.prim_algorithm(val_dep)
        
    
    # Dessin du labyrinthe
    def draw_maze(self) :
        # Affichage des noeuds
        for i in range(self.lines) :
            for j in range(self.columns) :
                value = i*self.columns + j
                
                # Le coefficient 0.7 est empirique                                   
                circle(self.screen.my_screen,self.graph.color,self.graph.nodes[value][0].coords,0.7*self.size)
           
        
        # Affichage des liens parents/enfants
        for id_parent in range(self.graph.nb_vertices) :
            parent = self.graph.nodes[id_parent][0]
                                                   
            for child in self.graph.nodes[id_parent] :
                id_child = child.value
                
        
                # Le parent est après l'enfant : rect horizontal partant de l'enfant                                   
                if id_parent - id_child == 1 :  
                    rect(self.screen.my_screen,self.graph.color,(child.coords[0]-self.size//2,
                                child.coords[1]-self.size//2,self.step_width + self.size//2,self.size))
                    
                # Le parent est avant l'enfant : rect horizontal partant du parent
                elif id_parent - id_child == -1 : 
                    rect(self.screen.my_screen,self.graph.color,(parent.coords[0]-self.size//2,
                                parent.coords[1]-self.size//2,self.step_width + self.size//2,self.size))
                    
                # Le parent est en-dessous de l'enfant : rect vertical partant de l'enfant    
                elif id_parent - id_child == self.columns : 
                    rect(self.screen.my_screen,self.graph.color,(child.coords[0]-self.size//2,
                                child.coords[1]-self.size//2,self.size,self.step_height + self.size//2))
                    
                # Le parent est au-dessus de l'enfant : rect vertical partant du parent    
                elif id_parent - id_child == -self.columns :  
                    rect(self.screen.my_screen,self.graph.color,(parent.coords[0]-self.size//2,
                                parent.coords[1]-self.size//2,self.size,self.step_height + self.size//2))
                
                # Dans les autres cas, on passe (parent sans enfant)
                else :
                    continue
               
            
    # Fin de l'application                
    def on_close(self) :
        self.event.on_close()

In [6]:
###########################################################################################
############################## DONNEES CONSTANTES #########################################
###########################################################################################

# Taille de l'écran. Ne pas dépasser ici 1200 (taille du fond d'écran chargé)
SCREEN_WIDTH = 600  # En longueur
SCREEN_HEIGHT = 600 # En largeur 

# Nombre de colonnes / Lignes du labyrinthe / Node de départ aléatoire (algorithme de Prim)
# Prendre des valeurs COHERENTES entières avec la taille de l'écran et supérieures ou égales à 3 : 
# SCREEN_WIDTH/SCREEN_HEIGHT = MAZE_COLUMNS/MAZE_LINES si possible, ce ne sera pas joli sinon :)
# ATTENTION : Choisir MAZE_COLUMNS >= 3 et MAZE_LINES >= 3
MAZE_COLUMNS = 20
MAZE_LINES = 20


####################### POUR TEST ! #####################################
DEPART_NODE = 0
#DEPART_NODE = randint(0,MAZE_LINES*MAZE_COLUMNS - 1)



# Couleur du labyrinthe. Code couleur Rouge Vert Bleu habituel. Entre 0 et 255 par valeur
# rouge / vert / bleu
MAZE_COLOR = (5,7,180)

# Génération au hasard. Valeur entière au moins égale à 2. Pas de différences sensibles 
# au-delà de 5, il s'agit des poids entre les sommets
MAZE_MAX_RANDOM = 5

###########################################################################################


# Lancement de la bibliothèque pygame 
pygame.init()
pygame.display.set_caption("Maze's generation")
            
# Création d'une fenêtre de dessin largeur / hauteur
screen = Screen(SCREEN_WIDTH,SCREEN_HEIGHT)
                
# Fond d'écran (taille maximale 1200 x 1200 environ pour celle chargée)
screen.set_image("Texture_Mur.jpg")


# Initialisation du labyrinthe
maze = Maze(screen,MAZE_LINES,MAZE_COLUMNS)
maze.set_size()
maze.set_color(MAZE_COLOR)
maze.set_max_weight(MAZE_MAX_RANDOM)

maze.init_maze()

################## POUR TEST !! ###############################
print("Répartition des poids dans le graphe : \n")
#maze.graph.print_graph()
###############################################################

# Algorithme de Prim
maze.graph.prim_algorithm(DEPART_NODE)

################## POUR TEST !! ###############################
print("\nArbre couvrant de poids minimal :\n ")
#maze.graph.print_graph()
###############################################################


# Affichage du résultat et gestion du "quit"
while screen.is_running :
    
    # Affichage de l'écran et du labyrinthe
    screen.blit()
    maze.draw_maze()
    
    # Affichage effectif
    flip()
    
    # Gestion de la fermeture de la fenêtre (clic sur la croix)
    maze.on_close()
    
    pygame.time.delay(20)  # Une image toutes les 20 ms  

Répartition des poids dans le graphe : 


Arbre couvrant de poids minimal :
 
